In [ ]:
# connect google drive to my colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import all libraries
import numpy as np
import keras
from keras.layers import Dense, Conv2D,MaxPool2D,Dropout,Flatten
from keras.models import Sequential 
from keras.preprocessing import image 

In [ ]:
# import data set 
# by data generator we can do image augmentations: multiple dimension for our model to learn 
train_datagen= image.ImageDataGenerator(
    rescale=1/255,horizontal_flip=True , zoom_range=0.2, shear_range=0.2)
# calling data set for training
train_data=train_datagen.flow_from_directory(directory="/content/drive/MyDrive/COVID-19_Radiography_Dataset/Train",
                                             target_size=(256,256), batch_size=16, class_mode="binary")

Found 5236 images belonging to 2 classes.


In [ ]:
# to show to classes : based on folder name 
train_data.class_indices

{'Covid': 0, 'Normal': 1}

In [ ]:
# rescale the image 
test_datagen= image.ImageDataGenerator(rescale=1/255)
test_data=test_datagen.flow_from_directory(directory="/content/drive/MyDrive/COVID-19_Radiography_Dataset/Val",
                                             target_size=(256,256), batch_size=16, class_mode="binary")

Found 1966 images belonging to 2 classes.


In [ ]:
# cnn model
model=Sequential()
# low level feature, relu for non linerality; 3 rgb
model.add(Conv2D(filters=32,kernel_size=(3,3),activation="relu",input_shape=(256,256,3)))
# increse filter
model.add(Conv2D(filters=64,kernel_size=(3,3),activation="relu"))
model.add(MaxPool2D())
model.add(Dropout(rate=0.25))

model.add(Conv2D(filters=128,kernel_size=(3,3),activation="relu"))
model.add(MaxPool2D())
model.add(Dropout(rate=0.25))

# flatten the layer image get flaten
# dense is multilayer perceptron

model.add(Flatten())
model.add(Dense(units= 64, activation="relu"))
# dropout to avoid overfitting
model.add(Dropout(rate=0.5))   
model.add(Dense(units=1, activation="sigmoid"))  # sigmoid softmax
model.compile(loss=keras.losses.binary_crossentropy,optimizer="adam",metrics=['acc'])   #  rmsprop adam

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 252, 252, 64)      18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 126, 126, 64)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 126, 126, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 124, 124, 128)     73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 128)      0         
 2D)                                                    

train model

In [ ]:
model.fit(train_data,steps_per_epoch=20, epochs=50 ,validation_steps=2, validation_data=test_data)

Epoch 1/50
20/20 [==============================] - 118s 5s/step - loss: 1.9661 - acc: 0.5125 - val_loss: 0.6936 - val_acc: 0.4375
Epoch 2/50
20/20 [==============================] - 91s 5s/step - loss: 0.6949 - acc: 0.4643 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 3/50
20/20 [==============================] - 93s 5s/step - loss: 0.6701 - acc: 0.6187 - val_loss: 0.6739 - val_acc: 0.6562
Epoch 4/50
20/20 [==============================] - 84s 4s/step - loss: 0.6325 - acc: 0.6469 - val_loss: 0.7051 - val_acc: 0.5000
Epoch 5/50
20/20 [==============================] - 85s 4s/step - loss: 0.6088 - acc: 0.6406 - val_loss: 0.6930 - val_acc: 0.5938
Epoch 6/50
20/20 [==============================] - 89s 4s/step - loss: 0.5551 - acc: 0.7188 - val_loss: 0.7526 - val_acc: 0.4062
Epoch 7/50
20/20 [==============================] - 84s 4s/step - loss: 0.5656 - acc: 0.7000 - val_loss: 0.7315 - val_acc: 0.6562
Epoch 8/50
20/20 [==============================] - 68s 3s/step - loss: 0.5554 - acc: 0.7

In [ ]:
path="/content/drive/MyDrive/COVID-19_Radiography_Dataset/Val/Normal/Normal-6999.png"
img=image.load_img(path,target_size=(256,256))
# conert image to number
img=image.img_to_array(img)/255
# 1D Array
img=np.array([img])

img.shape

(1, 256, 256, 3)

In [ ]:
# model.predict_classes(img)
(model.predict(img) > 0.5).astype("int32")

array([[1]], dtype=int32)

In [ ]:
path="/content/drive/MyDrive/COVID-19_Radiography_Dataset/Val/Covid/COVID-2619.png"
img=image.load_img(path,target_size=(256,256))
img=image.img_to_array(img)/255

img=np.array([img])

img.shape
(model.predict(img) > 0.5).astype("int32")

array([[0]], dtype=int32)